In [1]:
#Load dependencies
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine

#Define data files to import
houstonpopulationrawdata='raw_data/houston_population.csv'
houstonenergyemployeesrawdata='raw_data/houston_energy_employees.csv'
houstonemployeesrawdata='raw_data/houston_employees.csv'
housing_data = 'raw_data/houston_housing.csv'

#Read data files into Panda data frames
houstonenergyemployeesdata=pd.read_csv(houstonenergyemployeesrawdata,parse_dates=['DATE'])
houstonpopulationdata=pd.read_csv(houstonpopulationrawdata, parse_dates=['DATE'])
houstonemployeesdata=pd.read_csv(houstonemployeesrawdata,parse_dates=['DATE'])
housing_raw = pd.read_csv(housing_data, parse_dates=['DATE'])
oilprices = pd.read_html('https://www.macrotrends.net/1369/crude-oil-price-history-chart')
oilprices_df = oilprices[0]
oilprices_df = oilprices_df.droplevel(level=0, axis=1)

In [2]:
#Calculate employee data by year by using the mean of the monthly employee data
houstonemp_df=pd.DataFrame(houstonemployeesdata.groupby(houstonemployeesdata['DATE'].dt.year)['HOUS448NA'].mean()*1000)
houstonengyemp_df=pd.DataFrame(houstonenergyemployeesdata.groupby(houstonenergyemployeesdata['DATE'].dt.year)['SMU48264201021100001A'].mean()*1000)
housing_df = pd.DataFrame(housing_raw.groupby(housing_raw['DATE'].dt.year)['ATNHPIUS26420Q'].mean().reset_index())



In [3]:
#renaming columns and setting index for houston employees
houstonemp_df=houstonemp_df.rename_axis('year').rename(columns={'HOUS448NA':'employees'}).astype(int)
houstonengyemp_df=houstonengyemp_df.rename_axis('year').rename(columns={'SMU48264201021100001A':'employees'}).astype(int)

#renaming columns and setting index for houston population
houstonpopulationdata['year'] = houstonpopulationdata['DATE'].dt.year
houstonpopulationdata['population'] = houstonpopulationdata['HTNPOP'] * 1000
houstonpop_df = houstonpopulationdata[['year', 'population']]
houstonpop_df = houstonpop_df.set_index('year')
houstonpop_df['population'] = houstonpop_df['population'].astype(int)


#renaming columns and setting index for houston housing
housing_df = housing_df.rename(columns={"DATE": "year", "ATNHPIUS26420Q": "housing_price_index"})
housing_df = housing_df.set_index('year')

#renaming columns and setting index for oil prices
oilprices_df = oilprices_df[['Year', 'AverageClosing Price']]
oilprices_df = oilprices_df.rename(columns={"Year": "year", "AverageClosing Price": "oilprice_usd"})
oilprices_df = oilprices_df.set_index('year')
oilprices_df = oilprices_df.rename(columns={"Year": "year", "AverageClosing Price": "oilprice"})
oilprices_df ['oilprice_usd'] = oilprices_df['oilprice_usd'].str.replace('$',"")

In [4]:
#merged_df = pd.concat([oilprices_df,housing_df,houstonpop_df,houstonengyemp_df,houstonemp_df], sort=True)
merged1 = pd.merge(houstonemp_df, houstonengyemp_df, left_index=True, right_index=True, how='inner')
merged2 = pd.merge(merged1, housing_df, left_index=True, right_index=True, how='inner')
merged3 = pd.merge(merged2, oilprices_df, left_index=True, right_index=True, how='inner')
merged_df = pd.merge(merged3, houstonpop_df, left_index=True, right_index=True, how='inner')

In [5]:
stats_table = merged_df.to_html()

In [6]:
merged_df.to_html('data.html')